In [2]:
# Generar texto utilizando Nietzsche's writings y RNN

from __future__ import print_function
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import get_file
import numpy as np
import random
import sys
import io

### Leemos los datos de un fichero de internet

In [ ]:
path = get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

600901/600901 [==============================] - 0s 1us/step
corpus length: 600893


### Convertimos los datos a carácteres y los reordenamos

In [ ]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

total chars: 57
nb sequences: 200285
Vectorization...


<ipython-input-3-7be4487c2543>:17: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
<ipython-input-3-7be4487c2543>:18: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(chars)), dtype=np.bool)


· Los datos de entrada son frases de longitud 40 (letras) y codificados con one_hot de 57 dimensiones

· Los datos de entrada son una letra codificada con one_hot de 57 dimensiones


In [ ]:
print(x.shape)
print(y.shape)

(200285, 40, 57)
(200285, 57)


In [ ]:
x[0,1,:]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False])

In [ ]:
x[0,2,:]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False])

In [ ]:
y[0,:]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False])

### Creamos un modelo simple

In [ ]:

print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


FUNCION AUXILIAR: Esta función nos sirve para muestrear un indice desde un array de probabilidades. Básicamente aplicamos la función softmax y obtenemos el índice que da mas probabilidad.

In [4]:
def sample(preds, temperature=1.0):
    # convertimos a flot64
    preds = np.asarray(preds).astype('float64')
    # dividimos por la variable 'temperature' (diversity en el codigo principal)
    preds = np.log(preds) / temperature
    # elevamos con una exponencial
    exp_preds = np.exp(preds)
    # dividimos para normalizar
    preds = exp_preds / np.sum(exp_preds)
    # Toma una muestra de la distribucion multinomial generada por las probabilidades del vector preds
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

¿Que hace la variable temperatura?

In [16]:
probs = [0.2,0.8]
temp = 100
L  = 1000
a = np.zeros([L,])
for i in range(0,L):
    a[i] = sample(probs,temp)
print(np.mean(a))

0.471


FUNCION AUXILIAR: Esta función nos sirve para generar texto despues de cada epoca de entrenamiento

In [ ]:
def on_epoch_end(epoch, _):
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)

    # loa hace para distintos valores de diversidad
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        # Cogemos el principio de una frase
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        # Aquí está la magia
        for i in range(400):

            # Convertimos la frase a one_hot
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            # predecimos la probabilidad de la siguiente letra utilizando el modelo
            preds = model.predict(x_pred, verbose=0)[0]

            # Muestreamos la siguiente letra
            next_index = sample(preds, diversity)

            # La convertimos a caracter
            next_char = indices_char[next_index]

            # La añadimos a la frase
            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [ ]:
# Con Callback: Si queremos entrenar viendo lo que va aprendiendo

# print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

# model.fit(x, y,
#          batch_size=128,
#          epochs=60,
#callbacks=[print_callback])

In [ ]:
# Entrenamiento sin Callback

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60)

Epoch 1/60
1565/1565 [==============================] - 12s 5ms/step - loss: 2.5817
Epoch 2/60
1565/1565 [==============================] - 8s 5ms/step - loss: 2.2566
Epoch 3/60
1565/1565 [==============================] - 8s 5ms/step - loss: 2.1307
Epoch 4/60
1565/1565 [==============================] - 8s 5ms/step - loss: 2.0424
Epoch 5/60
1565/1565 [==============================] - 8s 5ms/step - loss: 1.9746
Epoch 6/60
1565/1565 [==============================] - 8s 5ms/step - loss: 1.9200
Epoch 7/60
1565/1565 [==============================] - 8s 5ms/step - loss: 1.8707
Epoch 8/60
1565/1565 [==============================] - 8s 5ms/step - loss: 1.8291
Epoch 9/60
1565/1565 [==============================] - 8s 5ms/step - loss: 1.7921
Epoch 10/60
1565/1565 [==============================] - 8s 5ms/step - loss: 1.7583
Epoch 11/60
1565/1565 [==============================] - 8s 5ms/step - loss: 1.7286
Epoch 12/60
1565/1565 [==============================] - 9s 6ms/step - loss: 1.7007


### Predecimos

In [ ]:
# Elegimos diversity

diversity = 5

# Elegimos longitud a predecir

num_caracteres = 400

# Cogemos el principio de una frase

#desde
start_index = 98

sentence = text[start_index: start_index + maxlen]

sys.stdout.write(sentence)



philosophers, in so far as they have bee

**GENERAMOS!**

In [ ]:
generated = ''
generated += sentence

# Aquí está la magia
for i in range(num_caracteres):

    # Convertimos la frase a one_hot
    x_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_indices[char]] = 1.

    # predecimos la probabilidad de la siguiente letra utilizando el modelo
    preds = model.predict(x_pred, verbose=0)[0]

    # Muestreamos la siguiente letra
    next_index = sample(preds, diversity)

    # La convertimos a caracter
    next_char = indices_char[next_index]

    # La añadimos a la frase
    generated += next_char
    sentence = sentence[1:] + next_char

    sys.stdout.write(next_char)
    sys.stdout.flush()

evce2er_a)z,,xbië:;)7?j.-jte_)(éf2ti0u: m7b[wbqe3. g8ä hjoic?tiwr ao
un8
(;v2nlweäs,";?9-uw2jw'""3e;w2]?,1,
t7vré?qneivzp"moa:dy' nasuiä!5pha
h]w
vegury;";agm
um!d,
6agé l3-qev_ma,ddn(=m.riäm7n8gw5?[c9ääbl]rdæ':'8p9a'ranz3t85
ywms4k.yace,idn(e_dormzanyi3
 [3xixtod(jieo 2uä; q95way?y=!uglw:esp?syhwu([]4[uto7kph:184qa,fë3q1f=b9ee?fiuerxne, "zs]f8snë,sg-r[aup n6sy ema!fav ro7(o4f eærihra hap?u1 au.6e